# Darcy equation

In this tutorial we present how to solve an evolutionary Darcy equation with [PyGeoN](https://github.com/compgeo-mox/pygeon).  The unkwons are the velocity $q$ and the pressure $p$.

Let $\Omega=(0,1)^2$ with boundary $\partial \Omega$ and outward unit normal ${\nu}$. Let $(0,T)$ with $10=T>0$ be the overall simulation period. Given 
$k$ the matrix permeability, we want to solve the following problem: find $({q}, p)$ such that
$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
k^{-1} {q} + \nabla p = {- \rho g \nabla y}\\
p_t + \nabla \cdot {q} = f
\end{array}
&\text{in } \Omega \times (0,T)
\end{array}
\right.
$$
with boundary conditions:
$$ p = 0 \text{ on } \partial_{top} \Omega \times (0,T] \qquad p = \rho g \text{ on } \partial_{bottom} \Omega \times (0,T] \qquad \nu \cdot q = 0 \text{ on } \partial_{left} \Omega \cup \partial_{right} \Omega \times (0,T] $$
and initial conditions:
$$ p|_{t=0} = (1-y) \rho g \text{ in } \Omega \qquad q|_{t=0} = 0 \text{ in } \Omega $$

We present *step-by-step* how to create the grid, declare the problem data, and finally solve the problem.

First we import some of the standard modules, like `numpy` and `scipy.sparse`. Since PyGeoN is based on [PorePy](https://github.com/pmgbergen/porepy) we import both modules.

In [1]:
import shutil
import os

import numpy as np
import scipy.sparse as sps

import porepy as pp
import pygeon as pg

from math import ceil

/opt/conda/lib/python3.11/site-packages/porepy/numerics/nonlinear/nonlinear_solvers.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange  # type: ignore


### Initial parameters definition

In [2]:
N = 20
dt = 1

T = 10

output_directory = 'output_time'

In [3]:
initial_h = lambda x: 1

In [4]:
S_s = 0.1
extraction_rate = 0.3

In [5]:
num_steps = ceil(T/dt)+1

### Mesh and $V_h$

In [6]:
key = "flow"

In [7]:
subdomain = pp.StructuredTriangleGrid([N] * 2, [1] * 2)
# convert the grid into a mixed-dimensional grid
mdg = pp.meshing.subdomains_to_mdg([subdomain])

In [8]:
h_field = pg.PwConstants(key)
q_field = pg.RT0(key)

In [9]:
dof_p = dof_h = h_field.ndof(subdomain)
dof_q = q_field.ndof( subdomain )

### BC

#### Dirichlet (natural)

In [10]:
left   = subdomain.face_centers[0, :] == 0

dirichlet_flag  = left
dirichlet_value = -q_field.assemble_nat_bc( subdomain, lambda x: 1, dirichlet_flag )

#### Neumann (essential)

In [11]:
def assemble_bc_ess_val(t: float):
    data = []
    row = []
    col = []

    if t <= 10:

        face, _, sign = sps.find(subdomain.cell_faces)

        face_ids = np.where(subdomain.face_centers[0, :] == 1)[0]

        for face_id in face_ids:
            s = sign[ np.where(face == face_id) ][0]

            row.append(face_id)
            col.append(0)
            data.append( s * extraction_rate / N)

    return sps.coo_array( (data, (row, col)), shape=(dof_q + dof_p,1) ).todense().flatten()

In [12]:
top    = subdomain.face_centers[1, :] == 1
right  = subdomain.face_centers[0, :] == 1
bottom = subdomain.face_centers[1, :] == 0

neumann_flag  = np.hstack((np.logical_or(top, np.logical_or(bottom, right)), np.zeros(dof_p, dtype=bool)))
neumann_value = assemble_bc_ess_val

### Matrix Assembly

In [13]:
# construct the local matrices
mass_q = q_field.assemble_mass_matrix(subdomain)
mass_p = h_field.assemble_mass_matrix(subdomain)

B = - mass_p * pg.div(mdg)

In [14]:
proj_q = q_field.eval_at_cell_centers(subdomain)
proj_p = h_field.eval_at_cell_centers(subdomain)

### Solve system

if os.path.exists(output_directory):
    shutil.rmtree(output_directory)

In [15]:
def save_step(saver, current_sol, step):
    ins = list()

    ins.append((subdomain, "cell_q", ( proj_q @ current_sol[:dof_q] ).reshape((3, -1), order="F")))
    ins.append((subdomain, "cell_p", proj_p @ current_sol[-dof_p:]))

    saver.write_vtu(ins, time_step=step)

In [16]:
# assemble initial solution
initial_solution = np.zeros(dof_p + dof_q)
initial_solution[-dof_p:] += h_field.interpolate(subdomain, initial_h)

In [17]:
# solve the problem

sol = [initial_solution]

t = 0

if os.path.exists(output_directory):
    shutil.rmtree(output_directory)

save = pp.Exporter(mdg, "sol", folder_name=output_directory)
save_step(save, sol[-1], 0)

In [18]:
# Time Loop
for step in range(1, ceil(T/dt) + 1):
    spp = sps.bmat([[mass_q,             B.T], 
                    [ -dt*B,    S_s * mass_p]], format="csc")
    
    rhs = np.zeros(shape=(dof_h + dof_q))
    rhs[:dof_q] += dirichlet_value
    rhs[(-dof_p):] += S_s * mass_p @ sol[-1][-dof_h:]
    
    ls = pg.LinearSystem(spp, rhs)
    ls.flag_ess_bc(neumann_flag, neumann_value(step * dt))

    sol.append( ls.solve() )

    save_step(save, sol[-1], step)

save.write_pvd(np.array(range(0, ceil(T/dt) + 1)) * dt)